# Get DWD CDC Station List for Climate Data

## 1. About the DWD Open Data Portal 

The data of the Climate Data Center (CDC) of the DWD (Deutscher Wetterdienst, German Weather Service) is provided on an **FTP server**. <br> **FTP** stands for _File Transfer Protocol_.

Open the FTP link ftp://opendata.dwd.de/climate_environment/CDC/ in your browser (copy-paste) and find our how it is structured hierarchically.

You can also open the link with **HTTPS** (Hypertext Transfer Protocol Secure): https://opendata.dwd.de/climate_environment/CDC/

**Download and read** the document https://opendata.dwd.de/climate_environment/CDC/Readme_intro_CDC_ftp.pdf

**Q1:** In which temporal resolutions are the time series provided?

**Q2:** What is the difference between _historical_ and _recent_ data also with respect to quality control?

**Q3:** Are all meteorological parameters provided at the same temporal resolution?


## 2. Download the Station Meta Data 

We are interested in observations with following properties:

1. The observations are taken in Germany.
1. It is climate data.
1. The temporal resolution is annual.
1. Use historial data, nt recent.


Download the corresonding station meta data file (description) from the FTP server. The file you have to download is named `KL_Jahreswerte_Beschreibung_Stationen.txt`. The elements of the file name denote:

* KL: Klima, Climate, 
* Jahreswerte: Annual Values, 
* Beschreibung: Description, 
* Stationen: Stations

**Q1:** Under with path (directory, folder) on the FTP server do you find the file?

**Q2:** The Python FTP client we use is provided through the library _ftplib_: <br>
https://pythonprogramming.net/ftp-transfers-python-ftplib/ <br>
How to you use it?

**Q3:** Look at the code below. In which folder is the data stored locally? What is are relative and absolute paths?

In [39]:
server = "opendata.dwd.de"
user = "anonymous"
passwd = ""
# COMPLETE THE PATH: dir = "/climate_environment/CDC/observations_germany/..."
dir = "/climate_environment/CDC/observations_germany/climate/annual/kl/historical/"
filename = "KL_Jahreswerte_Beschreibung_Stationen.txt"
localpath = "../data"

In [40]:
from ftplib import FTP

In [41]:
#domain name or server ip:
ftp = FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)
res = ftp.cwd(dir)
print(res)
ftp.dir()

230 Login successful.
250 Directory successfully changed.
-rw-r--r--    1 9261     15101     1197601 Oct 23 09:10 KL_Jahreswerte_Beschreibung_Stationen.txt
-rw-r--r--    1 9261     15101       13239 May 10 13:45 jahreswerte_KL_00001_19310101_19860630_hist.zip
-rw-r--r--    1 9261     15101       20305 May 10 13:45 jahreswerte_KL_00003_18510101_20110331_hist.zip
-rw-r--r--    1 9261     15101       16405 May 10 13:45 jahreswerte_KL_00044_19710301_20221231_hist.zip
-rw-r--r--    1 9261     15101       13960 May 10 13:45 jahreswerte_KL_00052_19730101_20011231_hist.zip
-rw-r--r--    1 9261     15101        9318 May 10 13:45 jahreswerte_KL_00061_19750701_19780831_hist.zip
-rw-r--r--    1 9261     15101        9874 May 10 13:45 jahreswerte_KL_00070_19730601_19860930_hist.zip
-rw-r--r--    1 9261     15101       15069 May 10 13:45 jahreswerte_KL_00071_19861101_20191231_hist.zip
-rw-r--r--    1 9261     15101       13087 May 10 13:45 jahreswerte_KL_00072_19781001_19950531_hist.zip
-rw-r--r--  

In [42]:
def grabFile(filename,localpath):
    localfile = open(localpath+"/"+filename, 'wb')
    ftp.retrbinary('RETR ' + filename, localfile.write, 1024)
    localfile.close()

In [43]:
grabFile(filename,localpath)

In [44]:
# Finally disconnect from the FPT Server
res = ftp.quit()
print(res)

221 Goodbye.


## 3. Read the Station Data into a Pandas Dataframe

The Station Data is in fixed column format. Pandas provides a reader for text files with fixed column width.  

Search the Pandas doc https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html for this fixed column reader. Learn how to use it and read the station data file into a dataframe.

Hint: Count the characters per column (column wdith) in a text editor.

### Extract column names and translate them from DE to EN.

In [45]:
# extract column names. They are in German (de)
file = open(localpath+"/"+filename,"r",encoding="latin")
r = file.readline()
file.close()
colnames_de = r.split()
colnames_de

['Stations_id',
 'von_datum',
 'bis_datum',
 'Stationshoehe',
 'geoBreite',
 'geoLaenge',
 'Stationsname',
 'Bundesland']

In [46]:
# translation dictionary
translate = \
{'Stations_id':'station_id',
 'von_datum':'date_from',
 'bis_datum':'date_to',
 'Stationshoehe':'altitude',
 'geoBreite': 'latitude',
 'geoLaenge': 'longitude',
 'Stationsname':'name',
 'Bundesland':'state'}

In [47]:
for h in colnames_de:
    print(translate[h])

station_id
date_from
date_to
altitude
latitude
longitude
name
state


In [48]:
# Pythonic
colnames_en = [translate[h] for h in colnames_de]
print(colnames_en)

['station_id', 'date_from', 'date_to', 'altitude', 'latitude', 'longitude', 'name', 'state']


### Read the formatted data with pd.read_fwf().

In [49]:
import pandas as pd

In [50]:
help(pd.read_fwf)

Help on function read_fwf in module pandas.io.parsers.readers:

read_fwf(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, colspecs: 'Sequence[tuple[int, int]] | str | None' = 'infer', widths: 'Sequence[int] | None' = None, infer_nrows: 'int' = 100, dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>, **kwds) -> 'DataFrame | TextFileReader'
    Read a table of fixed-width formatted lines into DataFrame.
    
    Also supports optionally iterating or breaking of the file
    into chunks.
    
    Additional help can be found in the `online docs for IO Tools
    <https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html>`_.
    
    Parameters
    ----------
    filepath_or_buffer : str, path object, or file-like object
        String, path object (implementing ``os.PathLike[str]``), or file-like
        object implementing a text ``read()`` function.The string could be a URL.
        Valid URL schemes include http, ftp, s3, and file. For file 

In [51]:
# Skip the first two rows and set the column names.
df = pd.read_fwf(localpath+"/"+filename,skiprows=2,names=colnames_en,encoding="latin")
df.head()

,station_id,date_from,date_to,altitude,latitude,longitude,name,state
0,1,19310101,19860630,478,47.8413,8.8493,Aach,Baden-Württemberg
1,3,18510101,20110331,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
2,44,19710301,20221231,44,52.9336,8.2370,Großenkneten,Niedersachsen
3,52,19730101,20011231,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
4,61,19750701,19780831,339,48.8443,12.6171,Aiterhofen,Bayern


In [36]:
# Better parse dates! Column 0 should be treated as index. It makes the later export with pd.to_csv() easier.
df = pd.read_fwf(localpath+"/"+filename,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0,encoding="latin")
df.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
1,1931-01-01,1986-06-30,478,47.8413,8.8493,Aach,Baden-Württemberg
3,1851-01-01,2011-03-31,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
44,1971-03-01,2022-12-31,44,52.9336,8.2370,Großenkneten,Niedersachsen
52,1973-01-01,2001-12-31,46,53.6623,10.1990,Ahrensburg-Wulfsdorf,Schleswig-Holstein
61,1975-07-01,1978-08-31,339,48.8443,12.6171,Aiterhofen,Bayern


In [37]:
df.shape

(1195, 7)

## 4. Export the dataframe as CSV file

Use semicolons as field delimiters.

In [38]:
# extract basename (Filename) without extension
import os
fname = os.path.splitext(filename)[0]
csvname = fname + ".csv"
print(csvname)

df.to_csv(localpath+"/"+csvname, sep =";")

KL_Jahreswerte_Beschreibung_Stationen.csv


## 5. Import the CSV as point vector layer into QGIS.

## 6. Download the zip-Archive with the Digital Administrative Boundaries


https://www.opengeodata.nrw.de/produkte/geobasis/vkg/dvg/dvg1/ (update)
dvg1_EPSG25832_Shape.zip

DVG: Digitale Verwaltungsgrenzen, DVG1 has more details than DVG2.

How to use the data:  https://www.opengeodata.nrw.de/produkte/geobasis/vkg/dvg/dvg1/Nutzerinformationen.pdf

Download the pdf and use Google Translate (GT) to translate the pdf (upload to GT).

https://www.bezreg-koeln.nrw.de/geobasis-nrw/produkte-und-dienste/verwaltungskarten-und-grenzen/digitale-verwaltungsgrenzen

## Homework: Create a Map in QGIS

Follow the tutorial http://www.qgistutorials.com/en/docs/3/making_a_map.html

In class we created a vector data layer (point shape file) with the coordinates of the DWD CDC climate stations from a CSV file we generated from the meta data file downloaded from the open data DWD FTP archive (yearly values, temperature).

Create a map of the DWD climate stations located in NRW. Use a shapefile of the NRW administrative boundaries.

Use the EPSG:28532 coordinate reference system (projection). We will learn later what it is.
